In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DataType

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "",
    "fs.azure.account.oauth2.client.secret": '',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"
}

dbutils.fs.mount(
    source = "abfss://tokyo-olympics-data@tokyoolympics2021data.dfs.core.windows.net/",
    mount_point = "/mnt/tokyo-olympics",
    extra_configs = configs
)


True

In [0]:
%fs
ls "mnt/tokyo-olympics"

path,name,size,modificationTime
dbfs:/mnt/tokyo-olympics/raw-data/,raw-data/,0,1742259591000
dbfs:/mnt/tokyo-olympics/transform-data/,transform-data/,0,1742259600000


In [0]:
spark

In [0]:
athletes = spark.read.format('csv').load('/mnt/tokyo-olympics/raw-data/athletes.csv', header=True)

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes = spark.read.format('csv').option("InferSchema", "true").load('/mnt/tokyo-olympics/raw-data/athletes.csv', header=True)
coaches = spark.read.format('csv').option("InferSchema", "true").load('/mnt/tokyo-olympics/raw-data/coaches.csv', header=True)
entries_gender = spark.read.format('csv').option("InferSchema", "true").load('/mnt/tokyo-olympics/raw-data/entriesgender.csv', header=True)
medals = spark.read.format('csv').option("InferSchema", "true").load('/mnt/tokyo-olympics/raw-data/medals.csv', header=True)
teams = spark.read.format('csv').option("InferSchema", "true").load('/mnt/tokyo-olympics/raw-data/teams.csv', header=True)

In [0]:
athletes.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:

entries_gender = (entries_gender
    .withColumn("Total", entries_gender["Total"].cast(IntegerType()))
    .withColumn("Female", entries_gender["Female"].cast(IntegerType()))
    .withColumn("Male", entries_gender["Male"].cast(IntegerType())))

In [0]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
# top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry", "Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# average number of entries by gender for each sport
average_entries_by_gender = entries_gender.withColumn('Avg_Female', entries_gender['Female'] / entries_gender['Total']).withColumn('Avg_Male', entries_gender['Male'] / entries_gender['Total']).select('Discipline', 'Avg_Female', 'Avg_Male').show()

+--------------------+-------------------+-------------------+
|          Discipline|         Avg_Female|           Avg_Male|
+--------------------+-------------------+-------------------+
|      3x3 Basketball|                0.5|                0.5|
|             Archery|                0.5|                0.5|
| Artistic Gymnastics|                0.5|                0.5|
|   Artistic Swimming|                1.0|                0.0|
|           Athletics| 0.4747672709456149| 0.5252327290543851|
|           Badminton|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|0.38461538461538464| 0.6153846153846154|
|          Basketball|                0.5|                0.5|
|    Beach Volleyball|                0.5|                0.5|
|              Boxing|0.35294117647058826| 0.6470588235294118|
|        Canoe Slalom|                0.5|                0.5|
|        Canoe Sprint| 0.4939759036144578| 0.5060240963855421|
|Cycling BMX Frees...| 0.5263157894736842|0.47368421052

In [0]:
athletes.write.option("header", "true").csv("/mnt/tokyo-olympics/transformed-data/athletes")

In [0]:
coaches.write.option("header", "true").csv("/mnt/tokyo-olympics/transformed-data/coaches")
entries_gender.write.option("header", "true").csv("/mnt/tokyo-olympics/transformed-data/entries_gender")
medals.write.option("header", "true").csv("/mnt/tokyo-olympics/transformed-data/medals")
teams.write.option("header", "true").csv("/mnt/tokyo-olympics/transformed-data/teams")